# Aligning forecasts and observations

1. Read some Niño 3.4 forecasts and observations from the IRI DL.
2. Given them time coordinates that xarray/pandas understands (not ideal)
3. Add a "valid_time" variable to the forecast dataset (also not idea)
4. Automagically align forecasts and obervations

In [1]:
import numpy as np
import xarray as xr
import pandas as pd

from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
nmme_url = 'https://iridl.ldeo.columbia.edu/SOURCES/.Models/.NMME/'
n34_url = 'Y/-5/5/RANGEEDGES/X/190/240/RANGEEDGES%5BX/Y%5Daverage/'

In [ ]:
#Observations 
# Construct the url pointing to the data
dl_loc = 'SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/'
time_str = 'T/(Jan%201980)/(Dec%202022)/RANGE/'
obs_url = 'https://iridl.ldeo.columbia.edu/' + dl_loc + time_str + n34_url + 'zlev/removeGRID/dods'
ds_obs = xr.open_dataset(obs_url, decode_times=False)

# replace DL time index with a pandas time index
N = len(ds_obs.T)
ds_obs['T'] = pd.date_range(start='1/1/1980', periods=N, freq='MS')
ds_obs = ds_obs.rename({'T': 'time'})
obsa = ds_obs.sst.groupby('time.month') - ds_obs.sst.groupby('time.month').mean('time')
obsa.plot(figsize=(12, 3))
plt.show()

In [ ]:
#GFDL-SPEAR
model = 'GFDL-SPEAR'
h_url = nmme_url + '.GFDL-SPEAR/.HINDCAST/.MONTHLY/.sst/' + n34_url + 'dods'
f_url = nmme_url + '.GFDL-SPEAR/.FORECAST/.MONTHLY/.sst/S/(Jan%202021)/last/RANGE/' + n34_url + 'dods'
h_ds = xr.open_dataset(h_url, decode_times=False)
f_ds = xr.open_dataset(f_url, decode_times=False)
ds = xr.concat([h_ds, f_ds], dim='S')

# replace DL time index with a pandas time index
N = len(ds.S)
ds['S'] = pd.date_range(start='1/1/1991', periods=N, freq='MS')

ds['ssta'] = ds.sst.groupby('S.month') - ds.sst.groupby('S.month').mean('S')
ds.ssta.mean('M').plot(x='S', figsize=(12, 3))
plt.title('Forecast')
plt.show()

In [ ]:
# add a valid time to the forecast dataset
valid_time = np.empty((len(ds.S), len(ds.L)), dtype='datetime64[ns]')
for lead in range(len(ds.L)):
    valid_time[:, lead] = [(ds.S.values[ii] + pd.DateOffset(months=lead)) for ii in range(len(ds.S))]

ds['valid_time'] = (('S', 'L'), valid_time)

If your observations and forecast have a valid_time and good time grids, you can start here.

In [ ]:
# add extre time to the obs (e.g., the future!)
extra_time = np.unique(ds.valid_time)
obsa = obsa.reindex({'time': extra_time})

# this is where the magic happens
ds['obsa'] = (('S', 'L'), obsa.sel(time=ds.valid_time).data)
ds

In [ ]:
ds.obsa.plot(x='S', figsize=(12, 3))
plt.title('Observations matching forecasts')
plt.show()

In [ ]:
error = ds.ssta.mean('M') - ds.obsa
error.plot(x='S', figsize=(12, 3))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(13, 4))
ax.plot(ds.valid_time[:, 0], ds.obsa[:, 0], color='gray', lw=4, label='obs')
ax.plot(ds.valid_time.T, ds.ssta.mean('M').T, 'salmon', lw=1, alpha=0.5)
plt.legend()
plt.show()

# Make a dataset with target and lead dimensions

In [ ]:
ds_TL = xr.concat([ds.sel(L=l).swap_dims({"S":"valid_time"}) for l in ds.L],"L")

In [ ]:
ds_TL.ssta.mean('M').plot(figsize=(12, 3));

In [ ]:
ds_TL.obsa.plot(figsize=(12, 3));